[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ai-forever/gigachain/blob/master/docs/docs/use_cases/tagging.ipynb)

# Классификация текста по меткам

Маркировка означает присвоение документу меток по различным темам, таким как:

- настроение;
- язык;
- стиль (формальный, неформальный и другие);
- охваченные темы;
- политическая направленность.

<Image img={require('../../static/img/tagging.png')} alt="Описание изображения" />

## Обзор

Маркировка включает компоненты:

* `функция` — например, [извлечение данных](/docs/tutorials/extraction).Маркировка использует [функции](https://openai.com/blog/function-calling-and-other-api-updates), чтобы указывать, как модель должна тегировать документ.
* `схема` — определяет, как нужно маркировать документ.

## Быстрый старт

Работа маркировки с помощью GigaChain продемонстрирована с помощью вызова инструментов OpenAI.
Для этого вам нужно использовать метод [`with_structured_output`](/docs/how_to/structured_output), который поддерживают модели OpenAI:

In [ ]:
%pip install --upgrade --quiet gigachain gigachain-openai

# Set env var OPENAI_API_KEY or load from a .env file:
# import dotenv
# dotenv.load_dotenv()

В схеме укажите Pydantic-модель с несколькими свойствами и их типами.

In [4]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI

tagging_prompt = ChatPromptTemplate.from_template(
    """
Extract the desired information from the following passage.

Only extract the properties mentioned in the 'Classification' function.

Passage:
{input}
"""
)


class Classification(BaseModel):
    sentiment: str = Field(description="The sentiment of the text")
    aggressiveness: int = Field(
        description="How aggressive the text is on a scale from 1 to 10"
    )
    language: str = Field(description="The language the text is written in")


# LLM
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0125").with_structured_output(
    Classification
)

tagging_chain = tagging_prompt | llm

In [6]:
inp = "Estoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos!"
tagging_chain.invoke({"input": inp})

Classification(sentiment='positive', aggressiveness=1, language='Spanish')

Если на выходе нужны данные формате JSON вы можете просто вызвать метод `.dict()`

In [13]:
inp = "Estoy muy enojado con vos! Te voy a dar tu merecido!"
res = tagging_chain.invoke({"input": inp})
res.dict()

{'sentiment': 'negative', 'aggressiveness': 8, 'language': 'Spanish'}

В примерах видно, что модель правильно интерпретирует заданные требования требования.

Результаты могут варьироваться, например, настроения могут быть на разных языках ('positive', 'enojado' и других).

## Повышение качества вывода

Для повышения качества вывода модели следует более точно определить схему данных.

В частности, вы можете определить:

- возможные значения для каждого свойства;
- описание, которое поможет убедиться, что модель правильно интерпретирует свойство;
- список свойства, которые нужно возвращать обязательно.

Используйте перечисления `enum`, чтобы уточнить Pydantic-модель, заданную в примере выше:

In [14]:
class Classification(BaseModel):
    sentiment: str = Field(..., enum=["happy", "neutral", "sad"])
    aggressiveness: int = Field(
        ...,
        description="describes how aggressive the statement is, the higher the number the more aggressive",
        enum=[1, 2, 3, 4, 5],
    )
    language: str = Field(
        ..., enum=["spanish", "english", "french", "german", "italian"]
    )

In [15]:
tagging_prompt = ChatPromptTemplate.from_template(
    """
Extract the desired information from the following passage.

Only extract the properties mentioned in the 'Classification' function.

Passage:
{input}
"""
)

llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0125").with_structured_output(
    Classification
)

chain = tagging_prompt | llm

Теперь ответы модели будут ограничены в соответствии с заданными значениями.

In [17]:
inp = "Estoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos!"
chain.invoke({"input": inp})

Classification(sentiment='happy', aggressiveness=1, language='spanish')

In [18]:
inp = "Estoy muy enojado con vos! Te voy a dar tu merecido!"
chain.invoke({"input": inp})

Classification(sentiment='sad', aggressiveness=5, language='spanish')

In [19]:
inp = "Weather is ok here, I can go outside without much more than a coat"
chain.invoke({"input": inp})

Classification(sentiment='neutral', aggressiveness=2, language='english')

<!--
[Трассировка LangSmith](https://smith.langchain.com/public/38294e04-33d8-4c5a-ae92-c2fe68be8332/r) позволяет нам заглянуть под капот:

<Image img={require('../../static/img/tagging_trace.png')} alt="Описание изображения" />
-->

<!--
### Смотрите также

* Вы можете использовать [метаданные тегировщика](/docs/integrations/document_transformers/openai_metadata_tagger) трансформатора документов для извлечения метаданных из документа LangChain.
* Это охватывает ту же базовую функциональность, что и цепочка тегирования, только применяется к документу LangChain.
-->